In [1]:
!pip install snowflake-connector-python
!pip install chromadb 

In [18]:
import snowflake.connector
import boto3
import json
import chromadb
from var import *

def query_endpoint_with_json_payload(encoded_json):
    client = boto3.client('runtime.sagemaker')
    response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/json', Body=encoded_json)
    return response

def parse_response_multiple_texts(query_response):
    model_predictions = json.loads(query_response['Body'].read())
    embeddings = model_predictions['embedding']
    return embeddings

# Chroma Vector database setup
client = chromadb.Client()
collection_name = "all-my-documents"

# Create collection (get_or_create_collection is used to avoid duplication)
collection = client.get_or_create_collection(collection_name)

# Snowflake connection parameters
# Establish a connection
conn_str = f"snowflake://{user}:{password}@{account}/{database}/{schema}?warehouse={warehouse}&role={role}"
conn = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema,
    role=role
)

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Formulate your SQL query
query = f'SELECT {column_to_print} FROM {table}'

# Execute the query
cursor.execute(query)

# Fetch all the results
results = cursor.fetchall()

# Extract text data from Snowflake results
text_data = [row[0] for row in results]

# Close the cursor and connection
cursor.close()
conn.close()

# SageMaker endpoint name
endpoint_name = 'jumpstart-dft-hf-textembedding-all-minilm-l6-snow'

# Set up the SageMaker runtime client
sagemaker_runtime = boto3.client('sagemaker-runtime')

# Create payload using the extracted text data
payload = {"text_inputs": text_data}

# Convert payload to JSON and encode it
encoded_payload = json.dumps(payload).encode('utf-8')

# Query the SageMaker endpoint with the JSON payload
query_response = query_endpoint_with_json_payload(encoded_payload)

# Parse the response and extract embeddings
embeddings = parse_response_multiple_texts(query_response)

# Add embeddings to ChromaDB collection
for text, embedding in zip(text_data, embeddings):
    # Add each document with its embedding and metadata to the collection
    collection.add(
        documents=[text],
        embeddings=[embedding],
        metadatas=[{"source": "snowflake"}],  # Add metadata for source identification
        ids=[f"doc_{text_data.index(text)}"],  # Use unique identifiers for each document
    )
    
# Assuming you have a 'collection' object created earlier in your code


Add of existing embedding ID: doc_0
Add of existing embedding ID: doc_0
Add of existing embedding ID: doc_1
Add of existing embedding ID: doc_0
Add of existing embedding ID: doc_3
Add of existing embedding ID: doc_4
Add of existing embedding ID: doc_5
Add of existing embedding ID: doc_6
Add of existing embedding ID: doc_7
Add of existing embedding ID: doc_8
Add of existing embedding ID: doc_9
Add of existing embedding ID: doc_10
Add of existing embedding ID: doc_11
Add of existing embedding ID: doc_12
Add of existing embedding ID: doc_13
Add of existing embedding ID: doc_14
Add of existing embedding ID: doc_15
Add of existing embedding ID: doc_16
Add of existing embedding ID: doc_17
Add of existing embedding ID: doc_18
Add of existing embedding ID: doc_19
Add of existing embedding ID: doc_20
Add of existing embedding ID: doc_21
Add of existing embedding ID: doc_22
Add of existing embedding ID: doc_23
Add of existing embedding ID: doc_24
Add of existing embedding ID: doc_25
Add of exist

In [19]:
# Specify the query texts and the number of results you want
query_texts = ["WHICH CITY IS GOOD FOR SHOPPING??"]
n_results = 2

# Perform the query on the ChromaDB collection
query_results = collection.query(query_texts=query_texts, n_results=n_results)

query_results 

# Access the results
#results = query_results["results"]

# Print or use the results as needed
#print(results)

{'ids': [['doc_66', 'doc_0']],
 'distances': [[1.2431838512420654, 1.2441734075546265]],
 'metadatas': [[{'source': 'snowflake'}, {'source': 'snowflake'}]],
 'embeddings': None,
 'documents': [["Seoul South Korea has the world's largest underground shopping mall COEX Mall",
   'City_facts']]}